# Parse

WNLI

In [17]:
import pandas as pd 
df = pd.read_csv("data/WNLI/train.tsv",delimiter='\t',header=None,names=['sentence0','sentence1','label'])
print('Number of training sentences: {:,}\n'.format(df.shape[0]))
df.sample(10)

Number of training sentences: 636



,sentence0,sentence1,label
59,Jane knocked on Susan's door but she did not a...,Jane did not answer.,0
375,The donkey wished a wart on its hind leg would...,The donkey wished a wart on its hind leg would...,0
95,The customer walked into the bank and stabbed ...,The customer was immediately taken to the hosp...,0
326,Grant worked hard to harvest his beans so he a...,"Later, he and Tatyana would shell them and coo...",1
540,The donkey wished a wart on its hind leg would...,The donkey wished a wart on its hind leg would...,0
390,When the sponsors of the bill got to the town ...,The opponents were very much in the minority.,0
28,Sam broke both his ankles and he's walking wit...,The crutches should be better.,0
297,The large ball crashed right through the table...,The table was made of styrofoam.,1
281,"Paul tried to call George on the phone, but he...",George wasn't successful.,0
237,Mama came over and sat down beside Alice. Gent...,Alice stroked her hair and let the child weep.,0


MRPC

In [1]:
import pandas as pd 
df = pd.read_csv("data/MRPC/train.tsv",delimiter='\t',header=None,names=['label','ID1','ID2','sentence0','sentence1'])
print('Number of training sentences: {:,}\n'.format(df.shape[0]))
df.sample(10)

Number of training sentences: 3,647



,label,ID1,ID2,sentence0,sentence1
2653,0,462816,462552,"Earlier Friday, Taiwan reported 55 new cases b...","Yesterday, Taiwan reported 65 new cases, its b..."
82,0,472952,472535,"But Mitsubishi Tokyo Financial (JP:8306: news,...","Sumitomo Mitsui Financial (JP:8316: news, char..."
3429,1,3001417,3001446,"Former U.S. Rep. Frank McCloskey, 64, died thi...",McCloskey died Sunday afternoon in his home af...
2690,1,1380723,1380525,"In Virginia, Mr. Kilgore, a Republican, accuse...","The Virginia attorney-general, Jerry Kilgore, ..."
2448,1,2694682,2694609,You know I have always tried to be honest with...,You know I have always tried to be honest with...
328,0,487684,487726,The euro tagged another record high against th...,The euro ros further into record territory on ...
1960,1,3030254,3030287,Intel plans to present a paper on its findings...,The breakthrough technology was presented by I...
288,1,3214303,3214275,"Marisa Baldeo stated, however, the authority's...","As of now, they are not supposed to wear anyth..."
1294,0,3225863,3225896,The American Express Corp. has pledged at leas...,"The city had requested federal funds, but with..."
1623,1,2613357,2613445,The condition is associated with heart disease...,Those with diabetes run the risk of severe com...


In [18]:
sentences = df.sentence0.values[1:]+df.sentence1.values[1:]

labels = df.label.values[1:]

print(labels)

['1' '1' '1' '0' '0' '0' '0' '1' '1' '0' '1' '0' '0' '0' '0' '0' '1' '0'
 '1' '0' '0' '0' '0' '1' '0' '0' '0' '1' '0' '0' '1' '1' '1' '1' '1' '0'
 '1' '1' '1' '1' '1' '0' '1' '1' '1' '0' '0' '0' '1' '0' '0' '1' '1' '0'
 '0' '1' '1' '0' '0' '0' '1' '1' '1' '1' '0' '1' '0' '1' '1' '0' '1' '1'
 '0' '1' '1' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '1' '0' '1' '1' '1'
 '1' '1' '1' '0' '1' '0' '0' '1' '1' '1' '1' '0' '0' '1' '0' '1' '1' '1'
 '1' '1' '0' '1' '1' '0' '0' '0' '0' '1' '0' '1' '1' '1' '1' '0' '1' '1'
 '0' '0' '0' '1' '1' '1' '1' '0' '1' '0' '1' '0' '1' '0' '0' '0' '1' '0'
 '1' '1' '1' '0' '1' '0' '0' '0' '0' '0' '0' '1' '1' '1' '1' '0' '1' '0'
 '0' '1' '1' '1' '1' '1' '1' '0' '1' '1' '1' '1' '1' '0' '1' '0' '0' '0'
 '1' '0' '0' '1' '1' '1' '0' '0' '1' '1' '1' '0' '1' '1' '1' '1' '1' '1'
 '0' '1' '0' '1' '1' '1' '1' '1' '0' '0' '0' '0' '0' '1' '1' '0' '1' '0'
 '0' '0' '0' '0' '0' '0' '0' '0' '0' '1' '1' '0' '1' '0' '1' '1' '1' '0'
 '1' '1' '0' '0' '0' '1' '0' '0' '0' '1' '0' '1' '1

In [19]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained("bert_uncased_localpath",do_lower_case = True)

# Tokenize Dataset

In [20]:
max_len = 0

for sent in sentences:
    input_ids = tokenizer.encode(sent, add_special_tokens = True)

    max_len = max(max_len , len(input_ids))

print('Max sentence length: ', max_len)

Max sentence length:  107


In [21]:
import torch 
input_ids = []
attention_masks =[]

for sent in sentences:
    encoded_dict = tokenizer.encode_plus(
                                        sent,
                                        add_special_tokens = True ,#add 'CLS'
                                        max_length = 128, 
                                        pad_to_max_length = True,   #pad & truncate all sentences to max_length
                                        return_attention_mask =True,  #construct attn. masks
                                        return_tensors = 'pt',     #return pytorch tensors
                                        truncation = True
    )
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = [int(label) for label in labels]
labels = torch.tensor(labels)

print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])

/opt/conda/envs/ST5/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2606: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Original:  I stuck a pin through a carrot. When I pulled the pin out, it had a hole.The carrot had a hole.
Token IDs: tensor([  101,  1045,  5881,  1037,  9231,  2083,  1037, 25659,  1012,  2043,
         1045,  2766,  1996,  9231,  2041,  1010,  2009,  2018,  1037,  4920,
         1012,  1996, 25659,  2018,  1037,  4920,  1012,   102,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,   

# Train and Validation Split

In [22]:
from torch.utils.data import TensorDataset , random_split

# combine the training inputs into a TensorDataset
dataset = TensorDataset(input_ids,attention_masks,labels)

train_size = int(0.9 * len(dataset))

val_size = len(dataset) - train_size

train_dataset , val_dataset =random_split(dataset,[train_size,val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

  571 training samples
   64 validation samples


In [23]:
from torch.utils.data import DataLoader,RandomSampler,SequentialSampler

batch_size = 32 

train_dataloader = DataLoader(train_dataset,
                              sampler= RandomSampler(train_dataset),
                              batch_size=batch_size)

validation_dataloader = DataLoader(val_dataset,
                                   sampler=RandomSampler(val_dataset),
                                   batch_size=batch_size)

# Train Out Classification Model

In [24]:
from transformers import BertForSequenceClassification , AdamW,BertConfig
model_path = "bert_uncased_localpath"
device = "cuda:1" if torch.cuda.is_available() else "cpu"
print(device)
config = BertConfig.from_pretrained(model_path)
model = BertForSequenceClassification.from_pretrained(model_path, 
                                                      config=config
                                                      )
model.to(device)

cuda:1


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert_uncased_localpath and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

# Optimizer & Learning Rate Scheduler

In [25]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5,
                  eps = 1e-8)

/opt/conda/envs/ST5/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [26]:
from transformers import get_linear_schedule_with_warmup

epochs = 2
PATH = "./wnli_finetuned/wnli_model.pth"
#PATH = "./mrpc_finetuned/mrpc_model.pth"
patience = 10
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer,num_warmup_steps=0,num_training_steps=total_steps)

# Training Loop

In [27]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [28]:
from tqdm import tqdm
def train() :
    stale = 0
    best_acc = 0.0
    for epoch in range(epochs):
        model.train()
        train_loss = []
        train_accs = []
        for batch in tqdm(train_dataloader):
            input_ids = batch[0].to(device)
            input_mask = batch[1].to(device)
            labels = batch[2].to(device)
            model.zero_grad()
            result = model(input_ids, 
                       token_type_ids=None, 
                       attention_mask=input_mask, 
                       labels=labels,
                       return_dict=True)
            loss = result.loss
            logits = result.logits
            loss.backward()
            optimizer.step()
            scheduler.step()
            logits = logits.detach().cpu().numpy()
            label_ids = labels.to('cpu').numpy()
            acc = flat_accuracy(logits , label_ids)
            train_loss.append(loss.item())
            train_accs.append(acc)
        train_loss = sum(train_loss) / len(train_loss)
        train_acc = sum(train_accs) / len(train_accs)

        print(f"[ Train | {epoch + 1:03d}/{epochs:03d} ] loss = {train_loss:.5f}, acc = {train_acc:.5f}")
        
        model.eval()
        valid_loss = []
        valid_accs = []

       
        for batch in tqdm(validation_dataloader):
            input_ids = batch[0].to(device)
            input_mask = batch[1].to(device)
            labels = batch[2].to(device) 
            with torch.no_grad():
                result = model(input_ids,
                                token_type_ids=None, 
                                attention_mask=input_mask,
                                labels=labels,
                                return_dict=True)
                loss = result.loss
                logits = result.logits
                logits = logits.detach().cpu().numpy()
                label_ids = labels.to('cpu').numpy()
                acc = flat_accuracy(logits , label_ids)
            valid_loss.append(loss.item())
            valid_accs.append(acc)
                
        valid_loss = sum(valid_loss) / len(valid_loss)
        valid_acc = sum(valid_accs) / len(valid_accs)

        print(f"[ Valid | {epoch + 1:03d}/{epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")

        if valid_acc > best_acc:
            print(f"Best model found at epoch {epoch}, saving model")
            torch.save(model.state_dict(),PATH)
            best_acc =valid_acc
            stale = 0
        else:
            stale += 1
            if stale > patience:
                print(f"No improvment {patience} consecutive epochs, early stopping")
                break
train()

100%|██████████| 18/18 [00:03<00:00,  5.81it/s]


[ Train | 001/002 ] loss = 0.70627, acc = 0.51286


100%|██████████| 2/2 [00:00<00:00, 20.09it/s]


[ Valid | 001/002 ] loss = 0.70909, acc = 0.42188
Best model found at epoch 0, saving model


100%|██████████| 18/18 [00:03<00:00,  5.97it/s]


[ Train | 002/002 ] loss = 0.69222, acc = 0.55279


100%|██████████| 2/2 [00:00<00:00, 20.13it/s]

[ Valid | 002/002 ] loss = 0.71512, acc = 0.37500


# Performance On Test(dev.dsv) Set

WNLI

In [29]:
import pandas as pd

df = pd.read_csv("data/WNLI/dev.tsv",delimiter='\t',header=None,names=['sentence0','sentence1','label'])

print('Number of test sentences: {:,}\n'.format(df.shape[0]))


sentences = df.sentence0.values[1:]+df.sentence1.values[1:]

labels = df.label.values[1:]

# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

for sent in sentences:
    encoded_dict = tokenizer.encode_plus(
                                        sent,
                                        add_special_tokens = True ,#add 'CLS'
                                        max_length = 128, 
                                        pad_to_max_length = True,   #pad & truncate all sentences to max_length
                                        return_attention_mask =True,  #construct attn. masks
                                        return_tensors = 'pt',     #return pytorch tensors
                                        truncation = True
    )
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = [int(label) for label in labels]
labels = torch.tensor(labels)

# Set the batch size.  
batch_size = 16 

# Create the DataLoader.
prediction_data = TensorDataset(input_ids, attention_masks, labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

Number of test sentences: 72



/opt/conda/envs/ST5/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2606: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


MRPC

In [15]:
import pandas as pd 
df = pd.read_csv("data/MRPC/dev.tsv",delimiter='\t',header=None,names=['label','ID1','ID2','sentence0','sentence1'])
print('Number of test sentences: {:,}\n'.format(df.shape[0]))

sentences = df.sentence0.values[1:]+df.sentence1.values[1:]

labels = df.label.values[1:]

# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

for sent in sentences:
    encoded_dict = tokenizer.encode_plus(
                                        sent,
                                        add_special_tokens = True ,#add 'CLS'
                                        max_length = 512, 
                                        pad_to_max_length = True,   #pad & truncate all sentences to max_length
                                        return_attention_mask =True,  #construct attn. masks
                                        return_tensors = 'pt',     #return pytorch tensors
                                        truncation = True
    )
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = [int(label) for label in labels]
labels = torch.tensor(labels)

# Set the batch size.  
batch_size = 16 

# Create the DataLoader.
prediction_data = TensorDataset(input_ids, attention_masks, labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

Number of test sentences: 398



/opt/conda/envs/ST5/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2606: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [30]:

model =  BertForSequenceClassification.from_pretrained(model_path, config=config)
device = "cuda:1" if torch.cuda.is_available() else "cpu"
model = model.to(device)
model.eval()
prediction_accs = []
for batch in tqdm(prediction_dataloader):
    input_ids = batch[0].to(device)
    input_mask = batch[1].to(device)
    labels = batch[2].to(device) 
    with torch.no_grad():
        result = model(input_ids,
                        token_type_ids=None, 
                        attention_mask=input_mask,
                        return_dict=True)
        logits = result.logits
        logits = logits.detach().cpu().numpy()
        label_ids = labels.to('cpu').numpy()
        acc = flat_accuracy(logits , label_ids)
    prediction_accs.append(acc)
        
prediction_acc = sum(prediction_accs) / len(prediction_accs)

print(f"Before finetune acc = {prediction_acc:.5f}")

model.load_state_dict(torch.load(PATH))
prediction_accs = []
for batch in tqdm(prediction_dataloader):
    input_ids = batch[0].to(device)
    input_mask = batch[1].to(device)
    labels = batch[2].to(device) 
    with torch.no_grad():
        result = model(input_ids,
                        token_type_ids=None, 
                        attention_mask=input_mask,
                        return_dict=True)
        logits = result.logits
        logits = logits.detach().cpu().numpy()
        label_ids = labels.to('cpu').numpy()
        acc = flat_accuracy(logits , label_ids)
    prediction_accs.append(acc)
        
prediction_acc = sum(prediction_accs) / len(prediction_accs)

print(f"After finetune acc = {prediction_acc:.5f}")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert_uncased_localpath and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 5/5 [00:00<00:00, 39.19it/s]


Before finetune acc = 0.47321


100%|██████████| 5/5 [00:00<00:00, 38.29it/s]

After finetune acc = 0.59643
